In [13]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import LabelEncoder

import pickle


In [3]:
df =pd.read_excel("../data/a2_Dataset_90Percent.xlsx")


In [4]:
df.head()

,ID,DemAffl,DemAge,DemClusterGroup,DemGender,DemReg,DemTVReg,LoyalClass,LoyalSpend,LoyalTime
0,140,10.0,76.0,C,U,Midlands,Wales & West,Gold,16000.00,4.0
1,620,4.0,49.0,D,U,Midlands,Wales & West,Gold,6000.00,5.0
2,868,5.0,70.0,D,F,Midlands,Wales & West,Silver,0.02,8.0
3,1120,10.0,65.0,F,M,Midlands,Midlands,Tin,0.01,7.0
4,2313,11.0,68.0,A,F,Midlands,Midlands,Tin,0.01,8.0


## Processing:

In [8]:
#make copy
dfc= df.copy()

In [9]:
dfc = dfc.drop(columns=['ID'])
dfc.head()

,DemAffl,DemAge,DemClusterGroup,DemGender,DemReg,DemTVReg,LoyalClass,LoyalSpend,LoyalTime
0,10.0,76.0,C,U,Midlands,Wales & West,Gold,16000.00,4.0
1,4.0,49.0,D,U,Midlands,Wales & West,Gold,6000.00,5.0
2,5.0,70.0,D,F,Midlands,Wales & West,Silver,0.02,8.0
3,10.0,65.0,F,M,Midlands,Midlands,Tin,0.01,7.0
4,11.0,68.0,A,F,Midlands,Midlands,Tin,0.01,8.0


In [10]:
# Nulls:
dfc['DemAffl']=dfc['DemAffl'].fillna(dfc['DemAffl'].mode()[0])
dfc['DemAge']=dfc['DemAge'].fillna(dfc['DemAge'].mode()[0])
dfc['DemClusterGroup']=dfc['DemClusterGroup'].fillna(dfc['DemClusterGroup'].mode()[0])
dfc['DemGender']=dfc['DemGender'].fillna(dfc['DemGender'].mode()[0])
dfc['DemReg']=dfc['DemReg'].fillna(dfc['DemReg'].mode()[0])
dfc['DemTVReg']=dfc['DemTVReg'].fillna(dfc['DemTVReg'].mode()[0])
dfc['LoyalTime']=dfc['LoyalTime'].fillna(dfc['LoyalTime'].mean())

In [11]:
dfc.isna().sum()

DemAffl            0
DemAge             0
DemClusterGroup    0
DemGender          0
DemReg             0
DemTVReg           0
LoyalClass         0
LoyalSpend         0
LoyalTime          0
dtype: int64

In [14]:
# encoding
number = LabelEncoder()

dfc['DemClusterGroup'] = number.fit_transform(dfc['DemClusterGroup'].astype('str'))
integer_mapping = {l: i for i, l in enumerate(number.classes_)}
print(integer_mapping)

dfc['DemGender'] = number.fit_transform(dfc['DemGender'].astype('str'))
integer_mapping = {l: i for i, l in enumerate(number.classes_)}
print(integer_mapping)

dfc['DemReg'] = number.fit_transform(dfc['DemReg'].astype('str'))
integer_mapping = {l: i for i, l in enumerate(number.classes_)}
print(integer_mapping)

dfc['DemTVReg'] = number.fit_transform(dfc['DemTVReg'].astype('str'))
integer_mapping = {l: i for i, l in enumerate(number.classes_)}
print(integer_mapping)

dfc['LoyalClass'] = number.fit_transform(dfc['LoyalClass'].astype('str'))
integer_mapping = {l: i for i, l in enumerate(number.classes_)}
print(integer_mapping)

{'A': 0, 'B': 1, 'C': 2, 'D': 3, 'E': 4, 'F': 5, 'U': 6}
{'F': 0, 'M': 1, 'U': 2}
{'Midlands': 0, 'North': 1, 'Scottish': 2, 'South East': 3, 'South West': 4}
{'Border': 0, 'C Scotland': 1, 'East': 2, 'London': 3, 'Midlands': 4, 'N East': 5, 'N Scot': 6, 'N West': 7, 'S & S East': 8, 'S West': 9, 'Ulster': 10, 'Wales & West': 11, 'Yorkshire': 12}
{'Gold': 0, 'Platinum': 1, 'Silver': 2, 'Tin': 3}


## Import model

In [15]:
loaded_model = pickle.load(open('../models/gradientboost.pkl', 'rb'))


## Make prediction:

In [19]:
y_pred = loaded_model.predict(dfc)
y_pred_proba = loaded_model.predict_proba(dfc)[:, 1]

## Save Predictions:

In [22]:
df_prediction_prob = pd.DataFrame(y_pred_proba, columns = ['prob_1'])

dfx=pd.concat([dfc,df_prediction_prob], axis=1)

dfx.to_excel("../data/BuyProb_90Percent.xlsx")

dfx.head()

,DemAffl,DemAge,DemClusterGroup,DemGender,DemReg,DemTVReg,LoyalClass,LoyalSpend,LoyalTime,prob_1
0,10.0,76.0,2,2,0,11,0,16000.00,4.0,0.115047
1,4.0,49.0,3,2,0,11,0,6000.00,5.0,0.078244
2,5.0,70.0,3,0,0,11,2,0.02,8.0,0.118211
3,10.0,65.0,5,1,0,4,3,0.01,7.0,0.128949
4,11.0,68.0,0,0,0,4,3,0.01,8.0,0.240484
